假设某电商网站存在两份数据events.csv，users.csv，分别记录了用户的访问、下单和购买信息以及用户的个人信息，下面从不同的分析场景举例来说明如何使用SQL引擎进行常用的用户行为分析。

events.csv的数据格式如下：
1,83,pay_order,2021-04-09 01:13:20,sp_19819,85.01
2,1,pay_order,2021-04-08 09:52:18,sp_55012,50.97
3,72,add_cart,2021-04-06 05:39:10,sp_1044,0.0
4,73,visit,2021-04-07 14:28:30,sp_14826,0.0
5,53,visit,2021-04-10 08:28:59,sp_38361,0.0
6,94,pay_order,2021-04-05 22:19:46,sp_79211,5.19

user.csv的数据内容如下：
1,2,z6s5g0duce,28,1,长春
2,2,t+nhd2scbv,18,1,南京
3,4,4u8dmz+xgh,27,1,长沙
4,1,r4y5ti16j+,60,1,广州
5,2,ilgt53hb7c,62,2,兰州

首先生成测试两份数据：events.csv和user.csv

In [8]:
from sql_data_generator import DataGenerator
user_num = 10000
event_num = 1000000
generator = DataGenerator()
generator.user_data_generation("./users.csv", user_num)
generator.event_data_generation("./events.csv", event_num, user_num)

初始化一下环境

In [6]:
import blackhole as bh
schema_events = '''
    event_id Int64,
    user_id Int32, 
    event Enum('visit'=1,'add_cart'=2,'pay_order'=3), 
    time DateTime,
    item_id String,
    fee Float32 
'''
schema_users = '''
    user_id Int32,
    equip Enum('android'=1,'ios'=2,'wm'=3,'pc'=4),
    user_name String,
    age Int8,
    gender Enum('男'=1,'女'=2),
    city String
'''
format = 'CSV'
table_events = 'events'
table_users = 'users'

建表并导入数据

In [9]:
bh.sql("create table if not exists {} ({}) Engine=MergeTree() order by tuple()".format(table_events, schema_events))
bh.sql("insert into table {} from infile '{}' format CSV".format(table_events, "./events.csv"))
bh.sql("create table if not exists {} ({}) Engine=MergeTree() order by tuple()".format(table_users, schema_users))
bh.sql("insert into table {} from infile '{}' format CSV".format(table_users, "./users.csv"))

查询日访问量(PV统计)

In [10]:
sql_pv = '''SELECT count() as "今日PV" FROM events as t
WHERE toDate(t.time)=today() AND t.event='visit' '''
bh.sql(sql_pv).show()

  今日PV
--------
   33360


日活用户量(UV统计)

In [11]:
sql_uv = '''SELECT count(DISTINCT t.user_id) as "今日UV" FROM events as t
WHERE toDate(t.time)=today() AND t.event='visit' '''
bh.sql(sql_uv).show()

  今日UV
--------
    9619


In [ ]:
查询最近7天日活

In [12]:
sql_7days_uv = '''SELECT toString(toDate(t.time)) as "日期", count(DISTINCT t.user_id) as "当日UV" FROM events as t
WHERE t.event='visit' AND toDate(t.time) BETWEEN today()-7 AND today()
GROUP BY toDate(t.time)'''
bh.sql(sql_7days_uv).show()

日期          当日UV
----------  --------
2021-12-13      9973
2021-12-14     10001
2021-12-15      9999
2021-12-16     10001
2021-12-17     10000
2021-12-18     10001
2021-12-19     10001
2021-12-20      9619


查询今天分时活跃数

In [13]:
sql_time_uv = '''SELECT toHour(t.time) as "时段", count(DISTINCT t.user_id) as "小时UV" FROM events as t
WHERE t.event='visit' AND toDate(t.time)=today()
GROUP BY toHour(t.time)'''
bh.sql(sql_time_uv).show()

  时段    小时UV
------  --------
     0      3264
     1      3289
     2      3264
     3      3302
     4      3215
     5      3243
     6      3243
     7      3228
     8      1652


查询每天上午 10 点至 11 点的下单用户数

In [14]:
sql_10_11_up = '''SELECT toString(toDate(t.time)) as "日期", count(DISTINCT t.user_id) as "下单用户数" FROM events as t
WHERE t.event='add_cart' AND EXTRACT(HOUR FROM t.time) IN (10,11)
GROUP BY toDate(t.time)'''
bh.sql(sql_10_11_up).show()

日期          下单用户数
----------  ------------
2021-12-13          5500
2021-12-14          5439
2021-12-15          5481
2021-12-16          5460
2021-12-17          5534
2021-12-18          5388
2021-12-19          5394


查询来自某个城市的用户有多少

In [15]:
sql_city_users = '''SELECT t.city as "城市", count(t.user_id) as "用户数" FROM users as t
GROUP BY t.city'''
bh.sql(sql_city_users).show()

城市        用户数
--------  --------
银川           232
拉萨           236
大连           247
苏州           243
海口           232
天津           256
福州           239
成都           232
厦门           230
南昌           271
三亚           253
长春           276
杭州           210
邯郸           253
无锡           268
上海           232
香港           241
南京           244
深圳           257
西安           243
贵阳           237
台北           227
太原           253
兰州           251
济南           254
重庆           244
南宁           269
北京           223
广州           217
青岛           210
昆明           213
宁波           261
合肥           258
长沙           256
武汉           223
郑州           250
沈阳           253
呼和浩特       250
乌鲁木齐       264
哈尔滨         259
石家庄         233


漏斗分析 visit（访问）—add_cart（下单）—pay_order（支付）（窗口期 48 小时且严格满足事件先后顺序

In [16]:
sql_vap_analyze = ''' SELECT count(DISTINCT t.user_id) as "全流程用户数" FROM
(
SELECT user_id, windowFunnel(172800)(time, event='visit',event='add_cart',event='pay_order') as level
FROM events
GROUP BY user_id
) as t WHERE t.level=3 '''
bh.sql(sql_vap_analyze).show()

  全流程用户数
--------------
         10001


统计连续3(n)天访问的用户数

In [17]:
sql_3days_continues_visit = ''' SELECT count(t2.user_id) as "连续3天访问用户数" FROM
(
SELECT user_id, windowFunnel(3)(t.dt, runningDifference(t.dt)=1, runningDifference(t.dt)=1) as level FROM
(
SELECT user_id, toDate(time) as dt FROM events ORDER BY user_id, time
) as t GROUP BY t.user_id
) as t2 WHERE t2.level=2 '''
bh.sql(sql_3days_continues_visit).show()

  连续3天访问用户数
-------------------
              10001


统计过去3天内浏览最多的3件商品

In [18]:
sql_top_sp_in_3_days = ''' SELECT topK(3)(t.item_id) as res FROM events as t WHERE t.event='visit' AND toDate(now()) - toDate(t.time) <=3 '''
bh.sql(sql_top_sp_in_3_days).show()

res
----------------------------------
['sp_80222','sp_31621','sp_60158']


统计过去3天内消费最多的3位用户

In [19]:
sql_top_user_in_3_days = ''' SELECT t.user_id, t.fees as res FROM
(
SELECT user_id, sum(fee) as fees from events WHERE toDate(now()) - toDate(events.time) <=3 GROUP BY user_id
) as t ORDER BY res DESC limit 3 '''
bh.sql(sql_top_user_in_3_days).show()

  user_id      res
---------  -------
     2509  72062.9
     9500  69679.9
     1501  68914


以上例子几乎涵盖了绝大部分用户行为分析的场景，当然用法不止上面这些，还有一些变通的用法，这里这是举例说明如何使用SQL引擎